# Project Title
Project Description

#### The notebook is structured as follows:
1. [Import and Prepare the Data](#data): We will read the price and volume data for the stocks belonging to the S&P 500 list. 

2. [Get the Strategy Performance](#performance): We will create a function to use it both in the train and test data.

3. [Backtest the Strategy Over All Stocks](#backtest): We will use the above function to backtest the strategy with the train data.

4. [Choose the Best Five Stocks](#choose): We will choose five stocks with the highest Sharpe ratio. These ratios will be computed in the previous section.

Stocks: TSLA, NVDA, PYPL

## Import Libraries

In [137]:
# For data manipulation and visualisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For technical indicators computation
import talib as ta
import yfinance as yf
import datetime as dt

# For filtering warnings
import warnings
warnings.filterwarnings("ignore")

# For strategy analytics
# import data_modules.quantra_analytics as qa

In [151]:
# Download Stock Data from Yahoo Finance 
tsla = yf.download('TSLA',"2022-01-01")
nvda = yf.download('NVDA',"2022-01-01")
pypl = yf.download('PYPL',"2022-01-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [152]:
# Create Function to store a new trade in the trade book
def trade(data, time, entry_time, entry_price):
    exit_price = data.loc[time, 'Close'] 
    # Calculate PnL
    pnl = round(exit_price - entry_price, 2)
    # Calculate trading cost due to fees
    trading_cost = exit_price * 0.0002 * 2
    pnl -= trading_cost
    trades = pd.DataFrame(columns = ['Position', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'PnL'])
    return trades.append({
        'Position': 'Long',
        'Entry Time' : entry_time,
        'Entry Price' : entry_price,
        'Exit Time' : time, 
        'Exit Price': exit_price, 
        'PnL' : pnl
    }, ignore_index=True)


# Create the function to backtest the strategy
def performance(stock):
    """
    Function used to backtest the strategy for each stock.
    """
    # Copy the close prices
    data = stock.copy()
    # Create the Bollinger bands
    data['upper'], data['middle'], data['lower'] = ta.BBANDS(data['Close'], timeperiod=20)
    # Create the band's width
    data['bandwidth'] = (data['upper'] - data['lower'])/data['middle']
    # Calculate the rolling width
    data['rolling_bandwidth'] = data['bandwidth'].rolling(21).mean()
    # Compute the condition 1
    data['condition1'] = data['bandwidth'] > data['rolling_bandwidth']
    # Compute the condition 2
    data['condition2'] = data['High'] > data['upper']
    # Compute the signal: First step
    data['long_signal'] = np.where(data['condition1'] & data['condition2'], 1, np.nan)
    # Compute the signal: Second step
    data['long_signal'] = np.where(data['Low'] < data['middle'], 0, data['long_signal'])
    # Fill the NaN values with previous observations
    data['long_signal'] = data['long_signal'].fillna(method = "ffill")
    # Define variables to keep track of current position, take profit and stop loss multiples of ATR 
    current_position = 0
    stop_loss_multiple = 1.5
    take_profit_multiple = 2.5
    # Calculate the Average True Range(ATR)
    data['ATR'] = ta.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)
    # Entry time and price of when we buy the stock
    entry_time = np.nan
    entry_price = np.nan
    
    # Create the trade book where all the trades get stored
    trade_book = pd.DataFrame(columns = ['Position', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'PnL'])

    # Create the loop to backtest
    for time in data.index:
        # Condition to go long
        if current_position == 0 and data.loc[time, 'long_signal'] == 1:
            current_position = 1
            entry_time = time
            entry_price = data.loc[time, 'Close']
        elif current_position == 1:
            # Specify the stop loss target as the max between a multiple of the ATR or the middle Bollinger band
            stop_loss = max(entry_price - data.loc[time, 'ATR'] * stop_loss_multiple, data.loc[time, 'middle'])
            # Specify the take profit target
            take_profit = entry_price + data.loc[time, 'ATR'] * take_profit_multiple
            # Condition to exit the Long position
            if data.loc[time, 'Close'] > take_profit or data.loc[time, 'Close'] < stop_loss:
                trade_book = pd.concat([trade_book, trade(data, time, entry_time, entry_price)])
                trade_book = trade_book.reset_index(drop = True)
                current_position = 0
            
    # Create the long_position column
    data['long_position'] = np.nan
    data.loc[data.index.isin(trade_book['Entry Time']), 'long_position'] = 1
    data.loc[data.index.isin(trade_book['Exit Time']), 'long_position'] = 0
    data['long_position'] = data['long_position'].fillna(method = 'ffill')
    # Compute the strategy returns
    data['returns'] = data['long_position'].shift(1) * data['Close'].pct_change()
    data = data.dropna()
    data['cumulative_returns%'] = ((1+data['returns']).cumprod()-1)*100
    # Only select dates in the 2022 calendar year
    # Return data frame containing all information 
    return data, trade_book


In [153]:
performance(pypl)

(                  Open        High        Low      Close  Adj Close    Volume  \
 Date                                                                            
 2022-07-28   85.540001   86.500000  83.349998  85.860001  85.860001  12476500   
 2022-07-29   86.519997   87.620003  84.820000  86.529999  86.529999  12115700   
 2022-08-01   87.050003   89.209999  85.269997  88.565002  88.565002  11710200   
 2022-08-02   87.110001   90.669998  86.910004  89.629997  89.629997  19673900   
 2022-08-03  101.139999  101.946999  97.269997  97.919998  97.919998  53766800   
 ...                ...         ...        ...        ...        ...       ...   
 2023-07-21   72.730003   73.620003  71.830002  72.989998  72.989998  25508500   
 2023-07-24   73.040001   73.970001  73.014999  73.690002  73.690002   8845400   
 2023-07-25   73.680000   74.300003  72.940002  72.959999  72.959999   8536000   
 2023-07-26   72.889999   73.760002  72.410004  73.430000  73.430000  11332500   
 2023-07-27   74

In [154]:
def plot(data, trades):   
    # Create the long_position column
    data['buy_signal'] = np.nan
    data['sell_signal'] = np.nan

    # Assign the entry price to the buy signal
    data.loc[data.index.isin(trades['Entry Time']), 'buy_signal'] = trades['Entry Price'].values

    # Assign the exit price to the sell signal
    data.loc[trades['Exit Time'], 'sell_signal'] = trades['Exit Price'].values

    # Plot the Positions and Close Prices
    plt.figure(figsize=(15, 7))
    plt.plot('Close', data=data, label='Close Price')
    plt.plot('upperband', data=data, label='Close Price')
    plt.plot('lowerband', data=data, label='Close Price')
    plt.plot('bandwidth', data=data, label='Close Price')
    plt.scatter(data.index, data['buy_signal'],
                color='green', marker='^', alpha=1, s=100)
    plt.scatter(data.index, data['sell_signal'],
                color='red', marker='v', alpha=1, s=100)
    dates = data.index.strftime('%Y-%m')

    # Set the title and axis labels
    plt.title('Close Prices and Positions', fontsize=16)
    plt.xlabel('Year-Month', fontsize=15)
    plt.ylabel('Prices ($)', fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc='upper left', prop={'size': 15})
    plt.show()

In [155]:
tsla_data, tsla_trade_book = performance(tsla)
tsla_trade_book

,Position,Entry Time,Entry Price,Exit Time,Exit Price,PnL
0,Long,2022-03-22,331.326660,2022-04-04,381.816681,50.337273
1,Long,2022-04-05,363.753326,2022-04-11,325.309998,-38.570124
2,Long,2022-07-21,271.706665,2022-08-03,307.396667,35.567041
3,Long,2022-08-04,308.633331,2022-08-09,283.333344,-25.413333
4,Long,2022-08-10,294.356659,2022-08-22,289.913330,-4.555965
5,Long,2023-01-27,177.899994,2023-02-09,207.320007,29.337072
6,Long,2023-02-10,196.889999,2023-03-02,190.899994,-6.066360
7,Long,2023-05-26,193.169998,2023-06-02,213.970001,20.714412
8,Long,2023-06-05,217.610001,2023-06-09,244.399994,26.692240
9,Long,2023-06-12,249.830002,2023-06-20,274.450012,24.510220


In [156]:
plot(tsla_data, tsla_trade_book)

ValueError: Must have equal len keys and value when setting with an iterable